# Operational Research Assignment 2025
## Optimal Warehouse Positioning
### Lolos Ioannis - 10674
### lolosioann@ece.auth.gr

In [31]:
import numpy as np
from gurobipy import Model, GRB

### Data Representation

The problem data includes:

- **12 candidate warehouse locations** (`warehouses` array), each with:
  - Fixed installation cost (in thousand €) (`fixed_cost`),
  - Maximum capacity (in tons) (`capacity`).
  
- **12 sales centers**, each with a specific demand (in tons) (`customers` and `demand` arrays).

- A **transport cost matrix**, where the entry $c_{ij}$ denotes the cost to fully serve the demand of sales center $j$ from warehouse $i$.

  - If delivery is infeasible, the cost is marked as ∞ and that route is **not considered** in the optimization model.


In [32]:
warehouses = list(range(12))
customers = list(range(12))
transport_cost = np.array([
    [100,  80,  50,  50,  60, 100, 120,  90,  60,  70,  65, 110],
    [120,  90,  60,  70,  65, 110, 140, 110,  80,  80,  75, 130],
    [140, 110,  80,  80,  75, 130, 160, 125, 100, 100,  80, 150],
    [160, 125, 100, 100,  80, 150, 190, 150, 130,  np.inf,  np.inf,  np.inf],
    [190, 150, 130,  np.inf,  np.inf,  np.inf, 180, 150,  50,  50,  60, 100],
    [200, 180, 150,  np.inf,  np.inf,  np.inf, 100, 120,  90,  60,  75, 110],
    [120,  90,  60,  70,  65, 110, 140, 110,  80,  80,  75, 130],
    [120,  90,  60,  70,  65, 110, 140, 110,  80,  80,  75, 130],
    [140, 110,  80,  80,  75, 130, 160, 125, 100, 100,  80, 150],
    [160, 125, 100, 100,  80, 150, 190, 150, 130,  np.inf,  np.inf,  np.inf],
    [190, 150, 130,  np.inf,  np.inf,  np.inf, 200, 180, 150,  np.inf,  np.inf,  np.inf],
    [200, 180, 150,  np.inf,  np.inf,  np.inf, 100,  80,  50,  50,  60, 100]
])
fixed_cost = [3500, 9000, 10000, 4000, 3000, 9000, 9000, 3000, 4000, 10000, 9000, 3500] # thousands of euros
capacity = [300, 250, 100, 180, 275, 300, 200, 220, 270, 250, 230, 180] # in tons
demand = [120, 80, 75, 100, 110, 100, 90, 60, 30, 150, 95, 120] # in tons

Transport cost to serve the full demand is not that useful, as we are able to split the demand of a customer between multiple warehouses. We will work with 
cost per-ton, which is a more flexible approach

In [33]:
# Convert transport cost to per-ton cost
cost_per_ton = np.zeros_like(transport_cost, dtype=float)

for i in warehouses:
    for j in customers:
        if transport_cost[i, j] == np.inf:
            cost_per_ton[i, j] = np.inf
        else:
            cost_per_ton[i, j] = transport_cost[i, j] / demand[j]

print("Cost per ton matrix:")
for row in cost_per_ton:
    print('\t'.join(f"{val:.2f}" for val in row))


Cost per ton matrix:
0.83	1.00	0.67	0.50	0.55	1.00	1.33	1.50	2.00	0.47	0.68	0.92
1.00	1.12	0.80	0.70	0.59	1.10	1.56	1.83	2.67	0.53	0.79	1.08
1.17	1.38	1.07	0.80	0.68	1.30	1.78	2.08	3.33	0.67	0.84	1.25
1.33	1.56	1.33	1.00	0.73	1.50	2.11	2.50	4.33	inf	inf	inf
1.58	1.88	1.73	inf	inf	inf	2.00	2.50	1.67	0.33	0.63	0.83
1.67	2.25	2.00	inf	inf	inf	1.11	2.00	3.00	0.40	0.79	0.92
1.00	1.12	0.80	0.70	0.59	1.10	1.56	1.83	2.67	0.53	0.79	1.08
1.00	1.12	0.80	0.70	0.59	1.10	1.56	1.83	2.67	0.53	0.79	1.08
1.17	1.38	1.07	0.80	0.68	1.30	1.78	2.08	3.33	0.67	0.84	1.25
1.33	1.56	1.33	1.00	0.73	1.50	2.11	2.50	4.33	inf	inf	inf
1.58	1.88	1.73	inf	inf	inf	2.22	3.00	5.00	inf	inf	inf
1.67	2.25	2.00	inf	inf	inf	1.11	1.33	1.67	0.33	0.63	0.83


### Mathematics of the Model

The objective is to minimize:

$$
\sum_{i=1}^{12} y_i \cdot(f_i + \sum_{j=1}^n c_{ij} x_{ij})
$$

Where:

- $ i \in \{1, \ldots, 12\} $ is the index to the candidate warehouse locations,
- $ j \in \{1, \ldots, 12\} $ is the index to the sales centers,
- $ f_i $: fixed cost of opening warehouse $ i $,
- $ c_{ij} $: transportation cost from warehouse $ i $ to sales center $ j $ (if feasible),
- $ x_{ij} $: quantity (in tons) delivered from warehouse $ i $ to sales center $ j $,
- $ y_i \in \{0,1\} $: binary variable indicating if warehouse $ i $ is open.


Subject to:

1. **Demand satisfaction**:
   $$
   \sum_{i=1}^{12} x_{ij} = d_j, \quad \forall j = 1, \ldots, 12
   $$

2. **Capacity constraints**:
   $$
   \sum_{j=1}^{12} x_{ij} \leq u_i y_i, \quad \forall i = 1, \ldots, 12
   $$

3. **Non-negativity and binary variables**:
   $$
   x_{ij} \geq 0, \quad y_i \in \{0,1\}
   $$

Where:

- $ d_j $: demand at sales center $ j $,
- $ u_i $: capacity of warehouse $ i $,

Impossible deliveries (marked ∞) are excluded by not allowing the corresponding $ x_{ij} $ variables in the model.

In [ ]:
model = Model("warehouse_location")

y = model.addVars(warehouses, vtype=GRB.BINARY, name="open")
x = model.addVars(warehouses, customers, vtype=GRB.CONTINUOUS, name="flow")

model.setObjective(
    sum(
        y[i] * (fixed_cost[i] + sum(cost_per_ton[i][j] * x[i, j] for j in customers 
        if cost_per_ton[i][j] < np.inf)) 
        for i in warehouses
        ),
    GRB.MINIMIZE
)


In [35]:
# Each customer must receive exactly their demand
for j in customers:
    model.addConstr(sum(x[i, j] for i in warehouses if cost_per_ton[i][j] < np.inf) == demand[j],
                    name=f"demand_{j}")

# Only open warehouses can ship, and can't exceed capacity
for i in warehouses:
    model.addConstr(sum(x[i, j] for j in customers if cost_per_ton[i][j] < np.inf) <= capacity[i] * y[i],
                    name=f"capacity_{i}")

# Disallow flows where cost is infinite
for i in warehouses:
    for j in customers:
        if cost_per_ton[i][j] == np.inf:
            model.addConstr(x[i, j] == 0)


### Display Results

In [36]:
model.setParam("OutputFlag", 0)  # silence output
model.optimize()

if model.status == GRB.OPTIMAL:
    print(f"\nTotal cost: {model.objVal:.2f} thousand euros\n")

    print("Opened Warehouses:")
    for i in warehouses:
        if y[i].X > 0.5:
            print(f"  Warehouse {i+1} (capacity: {capacity[i]})")

    print("\nFlow Table (tons from warehouses to customers):\n")
    
    # Print header
    header = ["W\\C"] + [f"C{j+1}" for j in customers]
    print("\t".join(header))

    # Print rows
    for i in warehouses:
        row = [f"W{i+1}"]
        for j in customers:
            flow = x[i, j].X
            val = f"{flow:.1f}" if flow > 1e-3 else "-"
            row.append(val)
        print("\t".join(row))

else:
    print("No feasible solution found.")



Total cost: 17929.23 thousand euros

Opened Warehouses:
  Warehouse 1 (capacity: 300)
  Warehouse 5 (capacity: 275)
  Warehouse 8 (capacity: 220)
  Warehouse 9 (capacity: 270)
  Warehouse 12 (capacity: 180)

Flow Table (tons from warehouses to customers):

W\C	C1	C2	C3	C4	C5	C6	C7	C8	C9	C10	C11	C12
W1	120.0	5.0	75.0	100.0	-	-	-	-	-	-	-	-
W2	-	-	-	-	-	-	-	-	-	-	-	-
W3	-	-	-	-	-	-	-	-	-	-	-	-
W4	-	-	-	-	-	-	-	-	-	-	-	-
W5	-	-	-	-	-	-	-	-	-	150.0	5.0	120.0
W6	-	-	-	-	-	-	-	-	-	-	-	-
W7	-	-	-	-	-	-	-	-	-	-	-	-
W8	-	75.0	-	-	45.0	100.0	-	-	-	-	-	-
W9	-	-	-	-	65.0	-	-	-	-	-	90.0	-
W10	-	-	-	-	-	-	-	-	-	-	-	-
W11	-	-	-	-	-	-	-	-	-	-	-	-
W12	-	-	-	-	-	-	90.0	60.0	30.0	-	-	-
